In [1]:
import pandas as pd
import numpy as np
import itertools
import os

In [2]:
fingers = ['1', '2', '3', '4', '5']

# Calculating valid sequences with rules

In [3]:
presses_per_finger= {
    1:1,
    2:2,
    3:1,
    4:2,
    5:1,
}

In [4]:
def check_illegal_seq(seq):
    #check runs of 3
    for pos in range(len(seq)- 2):
        if (int(seq[pos]) == int(seq[pos + 1]) + 1 and int(seq[pos+1]) == int(seq[pos+2]) + 1):
            return True
        if (int(seq[pos]) == int(seq[pos + 1]) - 1 and int(seq[pos+1]) == int(seq[pos+2]) - 1):
            return True
    
    #check double press
    
    for pos in range(len(seq) - 1):
        if(int(seq[pos]) == int(seq[pos+1])):
            return True
        

    for finger in fingers:
        if seq.count(finger) == 0:
            return True
        

    # # match presses per finger 
    # for finger in fingers:
    #     if seq.count(finger) != presses_per_finger[int(finger)]:
    #         return True
        
    return False

In [5]:
check_illegal_seq('1243542')

False

In [6]:
valid_sequences = []

# generate all possible sequences of 7 digits from 1 to 5
for seq in itertools.product(range(1,6), repeat=7):
    seq = ''.join(map(str, seq))
    if not check_illegal_seq(seq):
        valid_sequences.append(seq)

In [7]:
len(valid_sequences)

5218

# Generating Target Files

In [10]:
windowSizes = ['1', '2', '3', '7']

iti = 3000 
execTime = 10000 # msecs for each trial maximum
precueTime_interval = [600, 1000] # msecs for planning before movement 
hand = 2 #left or right hand
num_trained_seq = 2

total_sub_num = 20
num_sessions = 3
num_blocks_per_session = 12


In [11]:
def determine_trained_seq_for_subj(subj_num):
    np.random.seed(subj_num)
    # while True:
    #     selected_sequences = np.random.choice(valid_sequences, num_trained_seq, replace=False)
    #     if sum([selected_sequences[0][i] == selected_sequences[1][i] for i in range(7)]) <= 2 and selected_sequences[0][0] != selected_sequences[1][0]:
    #         return selected_sequences

    selected_sequences = np.random.choice(valid_sequences, num_trained_seq, replace=False)
    return selected_sequences


In [12]:
for subnum in range(total_sub_num):
    print(determine_trained_seq_for_subj(subnum))

['2423251' '3415213']
['1214532' '2451532']
['3254124' '3241513']
['1324353' '1413521']
['2125341' '3412151']
['2415313' '1453132']
['5312141' '4152413']
['1325451' '1451352']
['4524213' '4352541']
['4351325' '3542521']
['3512434' '3142451']
['1243545' '2153414']
['3241425' '3531423']
['2515342' '4215315']
['5241535' '5153423']
['2353145' '2524513']
['4532351' '1324534']
['5243135' '5421423']
['3245341' '5154243']
['2142523' '5143542']


In [13]:
trained_symbols = ['#', '$']
random_symbol = '%'

In [286]:
### target file headers: [subNum, hand, isTrain, symbol, seq, isMasked, execTime, iti, precueTime, windowSize]

for sub in range(total_sub_num):
    
    subNum = sub + 1
    hand = 2
    trained_seqs = determine_trained_seq_for_subj(subNum)
    num_rep_rand = 2


    for session in range(num_sessions):
        for blocknum in range(num_blocks_per_session):
            block = pd.DataFrame(columns=["subNum", "hand", "isTrain", "symbol", "seq", "isMasked", "execTime", "iti", "precueTime", "windowSize"])

            # Trained Trials
            for seq_idx, seq in enumerate(trained_seqs):
                symbol = trained_symbols[seq_idx]
                for windowSize in windowSizes:
                    #Memory only trial
                    precueTime = np.random.randint(precueTime_interval[0], precueTime_interval[1])
                    trial = pd.DataFrame([[subNum, hand, 1, symbol, seq, 1, execTime, iti, precueTime, windowSize]], columns = block.columns)
                    block = pd.concat([block, trial], ignore_index=True)
                    #Memory sensory trial
                    precueTime = np.random.randint(precueTime_interval[0], precueTime_interval[1])
                    trial = pd.DataFrame([[subNum, hand, 1, symbol, seq, 0, execTime, iti, precueTime, windowSize]], columns = block.columns)
                    block = pd.concat([block, trial], ignore_index=True)


            # Random Trials
            for windowSize in windowSizes:
                for _ in range(num_rep_rand):
                    while True:
                        seq = np.random.choice(valid_sequences)
                        if seq not in trained_seqs:
                            break
                    symbol = random_symbol
                    # Sensory only trial
                    precueTime = np.random.randint(precueTime_interval[0], precueTime_interval[1])
                    trial = pd.DataFrame([[subNum, hand, 0, symbol, seq, 0, execTime, iti, precueTime, windowSize]], columns = block.columns)
                    block = pd.concat([block, trial], ignore_index=True)

            block = block.sample(frac=1).reset_index(drop = True)
            block.to_csv(f'SMSH1_subj{subNum}_session{session+1}_block{blocknum+1}.tgt', index=False, sep='\t')




    
    
    
    

### Target files for the last session (perturbation):

In [22]:
digit_change = [2,3,4]
number_super_blocks = 2
number_blocks_per_super_block = 6
num_pre_blocks = 1
last_sessions = [4]

In [23]:
### target file headers: [subNum, hand, isTrain, isChanged, symbol, seq, isMasked, digitChangePos, digitChangeValue, execTime, iti, precueTime, windowSize]

for sub in range(total_sub_num):
    
    subNum = sub + 1
    hand = 2
    trained_seqs = determine_trained_seq_for_subj(subNum)
    num_rep_rand = 2


    ####### Pre test blocks #########
    for session in last_sessions:
        for blocknum in range(num_pre_blocks):
            block = pd.DataFrame(columns=["subNum", "hand", "isTrain", "isChanged", "symbol", "seq", "isMasked", "digitChangePos", "digitChangeValue", "execTime", "iti", "precueTime", "windowSize"])

            # Trained Trials
            isChanged = 0
            digit_change_pos = -1
            digit_change_value = 0
            for seq_idx, seq in enumerate(trained_seqs):
                symbol = trained_symbols[seq_idx]
                for windowSize in windowSizes:
                    #Memory only trial
                    precueTime = np.random.randint(precueTime_interval[0], precueTime_interval[1])
                    trial = pd.DataFrame([[subNum, hand, 1, isChanged, symbol, seq, 1, digit_change_pos, digit_change_value, execTime, iti, precueTime, windowSize]], columns = block.columns)
                    block = pd.concat([block, trial], ignore_index=True)
                    #Memory sensory trial
                    precueTime = np.random.randint(precueTime_interval[0], precueTime_interval[1])
                    trial = pd.DataFrame([[subNum, hand, 1, isChanged, symbol, seq, 0, digit_change_pos, digit_change_value, execTime, iti, precueTime, windowSize]], columns = block.columns)
                    block = pd.concat([block, trial], ignore_index=True)


            # Random Trials
            isChanged = 0
            digit_change_pos = -1
            digit_change_value = 0
            for windowSize in windowSizes:
                for _ in range(num_rep_rand):
                    while True:
                        seq = np.random.choice(valid_sequences)
                        if seq not in trained_seqs:
                            break
                    symbol = random_symbol
                    # Sensory only trial
                    precueTime = np.random.randint(precueTime_interval[0], precueTime_interval[1])
                    trial = pd.DataFrame([[subNum, hand, 0, isChanged, symbol, seq, 0, digit_change_pos, digit_change_value, execTime, iti, precueTime, windowSize]], columns = block.columns)
                    block = pd.concat([block, trial], ignore_index=True)

            block = block.sample(frac=1).reset_index(drop = True)
            block.to_csv(f'SMSH1_subj{subNum}_session{session}_block{blocknum+1}.tgt', index=False, sep='\t')



            

    for session in last_sessions:
        for super_block_num in range(number_super_blocks):
            block = pd.DataFrame(columns=["subNum", "hand", "isTrain", "isChanged", "symbol", "seq", "isMasked", "digitChangePos", "digitChangeValue", "execTime", "iti", "precueTime", "windowSize"])

            # Trained Trials
            isChanged = 0
            digit_change_pos = -1
            digit_change_value = 0
            for seq_idx, seq in enumerate(trained_seqs):
                symbol = trained_symbols[seq_idx]
                for windowSize in windowSizes:
                    #Memory only trial
                    precueTime = np.random.randint(precueTime_interval[0], precueTime_interval[1])
                    trial = pd.DataFrame([[subNum, hand, 1, isChanged, symbol, seq, 1, digit_change_pos, digit_change_value, execTime, iti, precueTime, windowSize]], columns = block.columns)
                    block = pd.concat([block, trial], ignore_index=True)
                    #Memory sensory trial
                    precueTime = np.random.randint(precueTime_interval[0], precueTime_interval[1])
                    trial = pd.DataFrame([[subNum, hand, 1, isChanged, symbol, seq, 0, digit_change_pos, digit_change_value, execTime, iti, precueTime, windowSize]], columns = block.columns)
                    block = pd.concat([block, trial], ignore_index=True)


            # Random Trials
            isChanged = 0
            digit_change_pos = -1
            digit_change_value = 0
            for windowSize in windowSizes:
                for _ in range(num_rep_rand):
                    while True:
                        seq = np.random.choice(valid_sequences)
                        if seq not in trained_seqs:
                            break
                    symbol = random_symbol
                    # Sensory only trial
                    precueTime = np.random.randint(precueTime_interval[0], precueTime_interval[1])
                    trial = pd.DataFrame([[subNum, hand, 0, isChanged, symbol, seq, 0, digit_change_pos, digit_change_value, execTime, iti, precueTime, windowSize]], columns = block.columns)
                    block = pd.concat([block, trial], ignore_index=True)


            block = pd.concat([block]*(number_blocks_per_super_block - 1), ignore_index=True)

            # Change digit trials
            isChanged = 1
            for seq_idx, seq in enumerate(trained_seqs):
                symbol = trained_symbols[seq_idx]
                for windowSize in windowSizes:
                    for digit_change_pos in digit_change:
                        finger_cp = fingers.copy()
                        finger_cp.remove(seq[digit_change_pos]) # excluding original digit
                        finger_cp.remove(seq[digit_change_pos - 1]) # excluding previous digit
                        if (seq[digit_change_pos - 1] != seq[digit_change_pos + 1]): # excluding next digit
                            finger_cp.remove(seq[digit_change_pos + 1])
                        digit_change_value = np.random.choice(finger_cp)
                        
                        precueTime = np.random.randint(precueTime_interval[0], precueTime_interval[1])
                        trial = pd.DataFrame([[subNum, hand, 1, isChanged, symbol, seq, 0, digit_change_pos, digit_change_value, execTime, iti, precueTime, windowSize]], columns = block.columns)
                        block = pd.concat([block, trial], ignore_index=True)
            
                        

            block = block.sample(frac=1).reset_index(drop = True)
            blocks = np.array_split(block, number_blocks_per_super_block)
            for block_idx, block in enumerate(blocks):
                block.to_csv(f'SMSH1_subj{subNum}_session{session}_block{super_block_num * number_blocks_per_super_block + block_idx + 1 + num_pre_blocks}.tgt', index=False, sep='\t')




    
    
    
    

/Users/amin/Library/Python/3.9/lib/python/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/amin/Library/Python/3.9/lib/python/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/amin/Library/Python/3.9/lib/python/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/amin/Library/Python/3.9/lib/python/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Use

### Correcting target files block number:


In [24]:
#file format: SMSH1_{subnum}_session{sessionnum}_block{blocknum}.tgt
target_file_path = './TargetFiles/'

#iterate over all target files
for file in os.listdir(target_file_path):
    if file.endswith('.tgt'):
        print(file)
        #extract subnum, sessionnum, blocknum
        sub_num = int(file.split('_')[1][4:])
        print(sub_num)
        session_num = int(file.split('_')[2][7:])
        print(session_num)
        block_num = int(file.split('_')[3][5:].split('.')[0])
        print(block_num)

        if session_num == 4:
            #update the block_num in file name
            new_block_num = (session_num-1) * num_blocks_per_session + block_num
            new_file_name = f'SMSH1_subj{sub_num}_session{session_num}_block{new_block_num}.tgt'
            print(new_file_name)
            os.rename(target_file_path + file, target_file_path + new_file_name)

        print('-------------------')

SMSH1_subj9_session4_block9.tgt
9
4
9
SMSH1_subj9_session4_block45.tgt
-------------------
SMSH1_subj1_session2_block14.tgt
1
2
14
-------------------
SMSH1_subj19_session1_block12.tgt
19
1
12
-------------------
SMSH1_subj13_session3_block30.tgt
13
3
30
-------------------
SMSH1_subj5_session1_block12.tgt
5
1
12
-------------------
SMSH1_subj20_session2_block15.tgt
20
2
15
-------------------
SMSH1_subj9_session3_block28.tgt
9
3
28
-------------------
SMSH1_subj7_session2_block18.tgt
7
2
18
-------------------
SMSH1_subj18_session4_block7.tgt
18
4
7
SMSH1_subj18_session4_block43.tgt
-------------------
SMSH1_subj12_session2_block16.tgt
12
2
16
-------------------
SMSH1_subj7_session2_block24.tgt
7
2
24
-------------------
SMSH1_subj19_session1_block3.tgt
19
1
3
-------------------
SMSH1_subj16_session1_block10.tgt
16
1
10
-------------------
SMSH1_subj15_session3_block28.tgt
15
3
28
-------------------
SMSH1_subj4_session2_block15.tgt
4
2
15
-------------------
SMSH1_subj8_session4_bl